In [13]:
import Main
from Analyser import *

In [14]:
modelName = 'Ruler'

In [15]:
# Some PyABC magic to get our set of parameters
from pyabc import (ABCSMC, Distribution, RV, MultivariateNormalTransition)
import scipy as sp
import scipy.stats as st
import tempfile
import os
import pandas as pd
import matplotlib.pyplot as plt

In [16]:
def model(params):
    return Main.generate_and_analyse(modelName, params)

In [17]:
metrics = {
    'section_1_w': (0.2, 1.4),
    'section_1_decay' : (0.4,1.0),
    'rule_1_0': (0,1),
    'rule_1_1': (0,1),
    'rule_1_2': (0,1),
    'rule_1_3': (0,1),
    'rule_1_4': (0,1),
}

In [18]:
prior = Distribution(section_1_w         =RV("uniform", 0.2, 1.4),  
                     section_1_decay     =RV("uniform", 0.4, 1.0),
                     rule_1_0            =RV("uniform", 0, 1),
                     rule_1_1            =RV("uniform", 0, 1),
                     rule_1_2            =RV("uniform", 0, 1),
                     rule_1_3            =RV("uniform", 0, 1),
                     rule_1_4            =RV("uniform", 0, 1),
                    )

In [19]:
abc = ABCSMC(model, prior, distance)

In [20]:
db_path = ("sqlite:///" + os.path.join(tempfile.gettempdir(), "test.db"))

In [21]:
target = Main.generate_and_analyse("Young_Shrub",{})

In [22]:
abc.new(db_path, target)

INFO:History:Start <ABCSMC(id=4, start_time=2019-04-16 00:14:41.712663, end_time=None)>
INFO:Epsilon:initial epsilon is 26.576443333319858


4

In [ ]:
history = abc.run(minimum_epsilon=0.1, max_nr_populations=10)

INFO:ABC:t:0 eps:26.576443333319858
INFO:ABC:t:1 eps:14.733328666364905
INFO:ABC:t:2 eps:12.287943656481364
INFO:ABC:t:3 eps:8.367633584215934
INFO:ABC:t:4 eps:6.778488866933616
INFO:ABC:t:5 eps:5.737424871688056
INFO:ABC:t:6 eps:5.144407698990144


In [ ]:
# Visualise results
from pyabc.visualization import plot_kde_1d
for (k,v) in metrics.items():
    fig, ax = plt.subplots()
    for t in range(history.max_t, history.max_t+1):
        df, w = history.get_distribution(m=0, t=t)
        plot_kde_1d(df, w,
                    xmin=v[0], xmax=v[1],
                    x=k, ax=ax,
                    label="PDF t={}".format(t))
    ax.legend();